In [28]:
import os
import numpy as np
import pandas as pd
from lxml import objectify
from sklearn.model_selection import train_test_split
import urllib

In [8]:
seed = 1783

In [2]:
def get_files(folder_path, ext=[], return_full_path=False):
    _files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f[-3:] in ext]
    if return_full_path:
        _files = [os.path.join(folder_path, f) for f in _files]
    return _files

In [3]:
d1 = get_files("./BarcodeDatasets/Dataset1", ext=["jpg", "JPG"], return_full_path=False)
d2 =  get_files("./BarcodeDatasets/Dataset2", ext=["jpg", "JPG"], return_full_path=False)
labelled_images = get_files("./labelled", ext=["jpg", "JPG"], return_full_path=True)
label_files = get_files("./labels", ext=["xml"], return_full_path=True)

In [37]:
annotations = {'image_name': [], 'x1': [], 'y1': [], 'x2': [], 'y2': [], 'class': [], 'dataset': []}
for _label_file in label_files:

    with open(_label_file, 'r') as file:
        anno = objectify.parse(_label_file).getroot()
        
    file_name = anno["filename"][0].text
    print(file_name)
    annotations['image_name'].append(os.path.join("./labelled", file_name))
    annotations['x1'].append(anno["object"][0]["bndbox"][0]["xmin"][0])
    annotations['y1'].append(anno["object"][0]["bndbox"][0]["ymin"][0])
    annotations['x2'].append(anno["object"][0]["bndbox"][0]["xmax"][0])
    annotations['y2'].append(anno["object"][0]["bndbox"][0]["ymax"][0])
    annotations['class'].append('barcode')
    if file_name in d1:
        if file_name in d2:
            raise ValueError("Filename match not found in both datasets: " + file_name)
        else:
            annotations['dataset'].append(1)
    elif file_name in d2:
        annotations['dataset'].append(2)
    else:
        raise ValueError("Filename match not found in either dataset: " + file_name)

05102009156.jpg
Foto(811).jpg
05102009146.jpg
Foto(840).jpg
Foto(790).jpg
PICT0020.JPG
Foto(765).jpg
Foto(708).jpg
Foto(477).jpg
05102009125.jpg
PIadwCT0019.JPG
PIfghghfCT0009.JPG
Foto(481).jpg
PICT00hfghg10.JPG
Foto(815).jpg
Foto(546).jpg
Foto(854).jpg
05102009102.jpg
06102009255.jpg
Foto(730).jpg
PICT0005.JPG
05102009105.jpg
Foto(848).jpg
05102009215.jpg
Foto(736).jpg
05102009175.jpg
Foto(686).jpg
Foto(783).jpg
Foto(687).jpg
05102009211.jpg
Foto(760).jpg
Foto(536).jpg
05102009199.jpg


In [38]:
print(annotations)

{'image_name': ['./labelled/05102009156.jpg', './labelled/Foto(811).jpg', './labelled/05102009146.jpg', './labelled/Foto(840).jpg', './labelled/Foto(790).jpg', './labelled/PICT0020.JPG', './labelled/Foto(765).jpg', './labelled/Foto(708).jpg', './labelled/Foto(477).jpg', './labelled/05102009125.jpg', './labelled/PIadwCT0019.JPG', './labelled/PIfghghfCT0009.JPG', './labelled/Foto(481).jpg', './labelled/PICT00hfghg10.JPG', './labelled/Foto(815).jpg', './labelled/Foto(546).jpg', './labelled/Foto(854).jpg', './labelled/05102009102.jpg', './labelled/06102009255.jpg', './labelled/Foto(730).jpg', './labelled/PICT0005.JPG', './labelled/05102009105.jpg', './labelled/Foto(848).jpg', './labelled/05102009215.jpg', './labelled/Foto(736).jpg', './labelled/05102009175.jpg', './labelled/Foto(686).jpg', './labelled/Foto(783).jpg', './labelled/Foto(687).jpg', './labelled/05102009211.jpg', './labelled/Foto(760).jpg', './labelled/Foto(536).jpg', './labelled/05102009199.jpg'], 'x1': [572, 418, 287, 426, 321

In [39]:
df = pd.DataFrame.from_dict(annotations, orient='columns')

In [40]:
df_train, df_val = train_test_split(df, test_size=0.33, random_state=seed, stratify=df['dataset'])

In [41]:
df_train.drop('dataset', axis=1).to_csv("annotations_train.csv", header=False, index=False)
df_val.drop('dataset', axis=1).to_csv("annotations_val.csv", header=False, index=False)

In [26]:
! retinanet-train \
csv ./annotations_train.csv ./classes.csv \
--val-annotations ./annotations_val.csv \
--random-transform \
--backbone resnet50 \
--imagenet-weights \
--compute-val-loss \
--image-max-side 512 \
--snapshot-path ./models/barcode_detector/

/bin/bash: retinanet-train: command not found


In [31]:
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
PRETRAINED_MODEL = 'models/pretrained_coco/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)

('models/pretrained_coco/resnet50_coco_best_v2.1.0.h5',
 <http.client.HTTPMessage at 0x7fb74a2fc310>)